## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [3]:
# answer below:
wine.info()
missing_data = wine.isnull().sum()/wine.shape[0]

missing_data.sort_values(ascending=False)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1599 entries, 1 to 1599
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed.acidity         1599 non-null   float64
 1   volatile.acidity      1599 non-null   float64
 2   citric.acid           1599 non-null   float64
 3   residual.sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free.sulfur.dioxide   1599 non-null   float64
 6   total.sulfur.dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 162.4 KB


quality                 0.0
alcohol                 0.0
sulphates               0.0
pH                      0.0
density                 0.0
total.sulfur.dioxide    0.0
free.sulfur.dioxide     0.0
chlorides               0.0
residual.sugar          0.0
citric.acid             0.0
volatile.acidity        0.0
fixed.acidity           0.0
dtype: float64

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [5]:
# answer below
wine['quality_ishigh'] = np.where((wine['quality']>5), 1, 0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,quality_ishigh
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


Create a train test split with 20% of the data in the test subsample.

In [7]:
# answer below:
Y = wine['quality_ishigh']
X = wine.drop(['quality', 'quality_ishigh'], 1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

Scale only the independent variables using the minmax scaler.

In [16]:
# answer below:
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [11]:
# answer below:
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)
y_pred_train = knn.predict(X_train)
y_pred_test = knn.predict(X_test)

print(f'Accuracy Train: {accuracy_score(y_train, y_pred_train)}')
print(f'Accuracy Test: {accuracy_score(y_test, y_pred_test)}')
print(f'F1 Train: {f1_score(y_train, y_pred_train)}')
print(f'F1 Test: {f1_score(y_test, y_pred_test)}')

Accuracy Train: 0.7756059421422987
Accuracy Test: 0.634375
F1 Train: 0.7891256429096252
F1 Test: 0.6811989100817438


In [17]:
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train_scaled, y_train)
y_pred_train = knn.predict(X_train_scaled)
y_pred_test = knn.predict(X_test_scaled)

print(f'Accuracy Train: {accuracy_score(y_train, y_pred_train)}')
print(f'Accuracy Test: {accuracy_score(y_test, y_pred_test)}')
print(f'F1 Train: {f1_score(y_train, y_pred_train)}')
print(f'F1 Test: {f1_score(y_test, y_pred_test)}')

Accuracy Train: 0.8100078186082877
Accuracy Test: 0.721875
F1 Train: 0.8250539956803455
F1 Test: 0.7561643835616437


**The results improved when using the scaled values.**

When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [100]:
# answer below:
test_dist = []
test_dist_sq = []

def inverse_sqrt(dist):
  test_dist.append(dist)
  with np.errstate(divide='ignore'):
    dist = 1. / (dist)**.5
  inf_mask = np.isinf(dist)
  inf_row = np.any(inf_mask, axis=1)
  dist[inf_row] = inf_mask[inf_row]
  test_dist_sq.append(dist)
  return dist

knn = KNeighborsClassifier(n_neighbors=5, weights=inverse_sqrt)

knn.fit(X_train, y_train)
y_pred_train = knn.predict(X_train)
y_pred_test = knn.predict(X_test)

print(f'Accuracy Train: {accuracy_score(y_train, y_pred_train)}')
print(f'Accuracy Test: {accuracy_score(y_test, y_pred_test)}')
print(f'F1 Train: {f1_score(y_train, y_pred_train)}')
print(f'F1 Test: {f1_score(y_test, y_pred_test)}')


Accuracy Train: 1.0
Accuracy Test: 0.7
F1 Train: 1.0
F1 Test: 0.7362637362637362


In [98]:
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')

knn.fit(X_train, y_train)
y_pred_train = knn.predict(X_train)
y_pred_test = knn.predict(X_test)

print(f'Accuracy Train: {accuracy_score(y_train, y_pred_train)}')
print(f'Accuracy Test: {accuracy_score(y_test, y_pred_test)}')
print(f'F1 Train: {f1_score(y_train, y_pred_train)}')
print(f'F1 Test: {f1_score(y_test, y_pred_test)}')


Accuracy Train: 1.0
Accuracy Test: 0.703125
F1 Train: 1.0
F1 Test: 0.7382920110192837
